here we are importing the dataset and performing data cleaning making the dataset suatiable for futher analysis.

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
df = pd.read_csv(r"C:\Users\sabni\Documents\SKILL\ObesityDataSet_raw_and_data_sinthetic.csv")
print(df.head(20))
print("\n\n\n\n")

# Drop missing values
df.dropna(axis=0, inplace=True)

# Display info and basic stats
print("DataFrame Info:")
print(df.info())
print("\n\n\n\n")

print("Descriptive Statistics:")
print(df.describe())
print("\n\n\n\n")

# Check for null values
print("Any Null Values Present:")
print(df.isnull().any())
print("\n\n\n\n")

print("Sum of Null Values per Column:")
print(df.isnull().sum())
print("\n\n\n\n")

# Check for duplicates
print("Any Duplicated Rows:")
print(df.duplicated().any())
print("\n\n\n\n")

# Drop duplicates
df.drop_duplicates(inplace=True)

# Save the cleaned DataFrame
output_path = r"C:\Users\sabni\Documents\SKILL\hi2.csv"
df.to_csv(output_path, index=False)



we are performing lable encoding by label encoder and one-hot-encoder
lable encoding is the feature in ml which allows us to convert the values in the categorical columns into numbers so that the ML model can better understand them and use them in analysis.

In [ ]:

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Copy original dataset for encoding
df_encoded = df.copy()

# Label Encoding for categorical columns
label_encoders = {}
for col in ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC','SMOKE', 
            'SCC', 'CALC', 'MTRANS','NObeyesdad']:
    le = LabelEncoder()
    df_encoded[col + '_Label'] = le.fit_transform(df_encoded[col])
    label_encoders[col] = le  # Store the encoder for reference

# One-Hot Encoding for categorical columns
one_hot_encoder = OneHotEncoder(sparse_output=False, drop='first')

one_hot_encoded = one_hot_encoder.fit_transform(df[['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC' ,'SMOKE',
            'SCC', 'CALC', 'MTRANS','NObeyesdad']])

# Convert One-Hot Encoded array to DataFrame
#one_hot_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.get_feature_names_out())

# Concatenate One-Hot Encoded columns with original dataframe
#df_encoded = pd.concat([df_encoded, one_hot_df], axis=1)
df=df_encoded.copy()
# Drop original categorical columns
df.drop(columns=['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC','SMOKE', 
            'SCC', 'CALC', 'MTRANS','NObeyesdad'], inplace=True)

# Display the first few rows of the encoded dataset
df.head(200)


here we are performing normailazation anf standerdization
Normalization (Min-Max Scaling): Rescales the data to a fixed range, typically [0,1] or [-1,1]. The formula is:
Standardization (Z-score Scaling): Transforms data to have a mean of 0 and a standard deviation of

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler


# Identifying numerical columns (ensure correct column names)
numerical_cols = ['Age','Height', 'Weight', 'FCVC', 'NCP', 'FAF', 'TUE']  # Removed 'Weight1'

# Min-Max Scaling (Normalization: scales values between 0 and 1)
min_max_scaler = MinMaxScaler()
df[[col + '_MinMax' for col in numerical_cols]] = min_max_scaler.fit_transform(df[numerical_cols])

# Standard Scaling (Z-score: mean = 0, std = 1)
standard_scaler = StandardScaler()
df[[col + '_ZScore' for col in numerical_cols]] = standard_scaler.fit_transform(df[numerical_cols])

# Display first few rows after scaling
print(df.head(10))


In [ ]:
# plotting a histogram of the output distribution 
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Plot histogram
plt.figure(figsize=(20, 6))
sns.histplot(df['NObeyesdad_Label'], kde=True, bins=30)
plt.title('Output Distribution')
plt.xlabel('NObeyesdad')
plt.ylabel('Frequency')
plt.show()


In [ ]:

# Plot 2: Histogram of Heart Rate with KDE
plt.figure(figsize=(20, 6))
sns.histplot(df['Age'], kde=True, bins=30)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# scatterplot of weight1 with weight
plt.figure(figsize=(10, 6))
sns.scatterplot(data =df, x=df['Weight'], y=df['Height'], hue="NObeyesdad_Label",palette="coolwarm")  # Scatter plot for two continuous variables
plt.title('Weight1 vs Height')
plt.xlabel('Weight')
plt.ylabel('Height')
plt.show()


In [ ]:
# Plot 6: Bar plot of Age vs.NObeyesdad_Label
plt.figure(figsize=(10, 6))
sns.barplot(x='Age', y="NObeyesdad_Label", data=df)  # ci=None to avoid confidence intervals
plt.title('Age vs.NObeyesdad_Label')
plt.xlabel('Age')
plt.ylabel('NObeyesdad_Label')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()

In [ ]:
#Plot 8: lineplot with marker of Age vs.NObeyesdad_Label
plt.figure(figsize=(10,6))
sns.lineplot(data=df, x="Age",y="NObeyesdad_Label", label="lineplot with marker", color="blue", marker="o" )
plt.title('Age vs NObeyesdad_Label')
plt.xlabel('Age')
plt.ylabel('NObeyesdad_Label')
plt.show()
# Count the occurrences of each unique value in the 'Heart_Attack_Risk' column
risk_counts = df['NObeyesdad_Label'].value_counts()


In [ ]:
# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(risk_counts, labels=risk_counts.index, autopct='%1.1f%%', startangle=90, colors=['lightblue', 'salmon','yellow'])
plt.title('NObeyesdad_Label')
plt.show()


Here, we are finding and removing outliers using the IQR and Z-score methods.  

Outliers are data points that deviate from the normal distribution of the data. These may reduce the accuracy of the model; hence, they are detected and removed before the dataset is given to the model for training.

In [ ]:
import pandas as pd



# Function to remove outliers using IQR
def remove_outliers_iqr(df):
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:  # Only process numerical columns
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Filter out outliers
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

    return df

# Apply the function to remove outliers
df_no_outliers_iqr = remove_outliers_iqr(df)


print("Original DataFrame:")
print(df)
print("\n\n\n\n")
print("\nDataFrame after removing outliers:")
print(df_no_outliers_iqr.head(20))
print("\n\n\n\n")
output_path = r"C:/Users/sabni/Documents/SKILL/hi3.csv"
df_no_outliers_iqr.to_csv(output_path, index=False)



In [ ]:
from scipy.stats import zscore

# Calculate Z-scores for each numeric column
for col in df.columns:
    if df[col].dtype in ['int64', 'float64']:
        z_scores = zscore(df[col])
        print(f"Z-scores for column '{col}':")
        print(z_scores)


# Function to remove outliers using Z-score
def remove_outliers_zscore(df, threshold=1):
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:  # Only process numerical columns
            z_scores = zscore(df[col])
            df = df[abs(z_scores) <= threshold]

    return df

# Apply the function to remove outliers
df_no_outliers_zscore = remove_outliers_zscore(df)

output_path = r"C:/Users/sabni/Documents/SKILL/hi4.csv"
df_no_outliers_zscore.to_csv(output_path, index=False)
print("\nDataFrame after removing outliers:")
print(df_no_outliers_zscore.head(20))

here we are performing feature selection using chi square method and ANOVA 
ANOVA compares the means of three or more groups to check if they differ significantly. It analyzes the effect of categorical independent variables on a continuous dependent variable using the F-statistic.
Chi-Square tests the association between two categorical variables by comparing observed and expected frequencies.

In [ ]:
from sklearn.feature_selection import f_classif, chi2
import pandas as pd


# Define feature sets
numerical_features = ["Age", "Height", "Weight1", "FCVC", "NCP", "FAF", "TUE", "Weight"]
categorical_features = ["Gender_Label", "family_history_with_overweight_Label", "FAVC_Label", "CAEC_Label", "SCC_Label", "CALC_Label", "MTRANS_Label"]

y = df["NObeyesdad_Label"]  # Using pre-encoded target variable

# ANOVA F-test for numerical features
X_numerical = df[numerical_features]
anova_results = f_classif(X_numerical, y)
anova_scores = pd.DataFrame({"Feature": numerical_features, "ANOVA F-Value": anova_results[0]})

# Chi-square test for pre-encoded categorical features
X_categorical = df[categorical_features]
chi2_results = chi2(X_categorical, y)
chi2_scores = pd.DataFrame({"Feature": categorical_features, "Chi-Square Score": chi2_results[0]})

# Select features based on threshold values
anova_threshold = 1.0  # Example threshold for ANOVA
chi2_threshold = 1.0  # Example threshold for Chi-Square

selected_numerical_features = anova_scores[anova_scores["ANOVA F-Value"] > anova_threshold]["Feature"].tolist()
selected_categorical_features = chi2_scores[chi2_scores["Chi-Square Score"] > chi2_threshold]["Feature"].tolist()

# Print results in tabular form
print("ANOVA F-Test Scores (Numerical Features):")
print(anova_scores.to_string(index=False))
print("\nChi-Square Scores (Categorical Features):")
print(chi2_scores.to_string(index=False))

# Print selected features
print("\nSelected Numerical Features based on ANOVA:")
print(selected_numerical_features)
print("\nSelected Categorical Features based on Chi-Square:")
print(selected_categorical_features)


Information Gain (IG) is a feature selection technique used in machine learning to determine the relevance of a feature based on how much it reduces uncertainty (entropy) in the target variable.
A higher Information Gain indicates a more important feature. It is widely used in decision trees and classification tasks to select the most informative attributes.

In [ ]:
from sklearn.feature_selection import mutual_info_classif
import pandas as pd

# Information Gain (Mutual Information) for all features
X_all = df[numerical_features + categorical_features]
info_gain_scores = mutual_info_classif(X_all, y)
info_gain_df = pd.DataFrame({"Feature": numerical_features + categorical_features, "Information Gain": info_gain_scores})

# Print Information Gain scores in tabular form
print("\nInformation Gain (Mutual Information) Scores:")
print(info_gain_df.to_string(index=False))
# Select features with Information Gain above a threshold (e.g., 0.01)
threshold = 0.2
selected_features = info_gain_df[info_gain_df["Information Gain"] > threshold]["Feature"].tolist()

# Print selected features
print("\nSelected Features based on Information Gain:")
print(selected_features)


Pearson’s Correlation Matrix
A Pearson’s Correlation Matrix is a table that shows the Pearson correlation coefficients between multiple variables. The Pearson correlation coefficient (r) measures the linear relationship between two numerical variables, ranging from -1 to 1:

r = 1 → Perfect positive correlation
r = -1 → Perfect negative correlation
r = 0 → No correlation
The matrix helps identify relationships between features, aiding in feature selection by removing highly correlated (redundant) variables in machine learning.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Compute Pearson's correlation matrix
correlation_matrix = df.corr(method='pearson')

# Display the correlation matrix
print("Pearson's Correlation Matrix:")
print(correlation_matrix.to_string())

# Pearson's Correlation Matrix for feature selection
correlation_matrix = df[numerical_features + categorical_features].corr().abs()

# Define a threshold for high correlation
correlation_threshold = 0.8
high_correlation_features = set()

for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if correlation_matrix.iloc[i, j] > correlation_threshold:
            high_correlation_features.add(correlation_matrix.columns[i])

# Selecting features that are not highly correlated
selected_features_pearson = [feature for feature in (numerical_features + categorical_features) if feature not in high_correlation_features]

# Print selected features
print("\nSelected Features based on Pearson's Correlation:")
print(selected_features_pearson)

# Visualize Pearson's Correlation Matrix
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Pearson's Correlation Matrix Heatmap")
plt.show()


Random Forest Feature Selection Method
Random Forest, an ensemble learning method, provides an effective way to select important features based on how much they contribute to decision-making.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Define features and target variable
X = df[numerical_features + categorical_features]
y = df["NObeyesdad_Label"]  # Target variable

# Train a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X, y)

# Get feature importances
feature_importances = pd.DataFrame({
    "Feature": X.columns,
    "Importance": rf_model.feature_importances_
})

# Sort features by importance
feature_importances = feature_importances.sort_values(by="Importance", ascending=False)

# Print feature importances
print("Feature Importance from Random Forest:")
print(feature_importances.to_string(index=False))

# Select top features based on a threshold
importance_threshold = 0.01  # Example threshold
selected_features_rf = feature_importances[feature_importances["Importance"] > importance_threshold]["Feature"].tolist()

print("\nSelected Features based on Random Forest Importance:")
print(selected_features_rf)


In [ ]:
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier

# Define features and target variable
X = df[numerical_features + categorical_features]
y = df["NObeyesdad_Label"]  # Target variable

# Train a Random Forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X, y)

# Perform permutation importance
perm_importance = permutation_importance(rf_model, X, y, scoring='accuracy', n_repeats=10, random_state=42)

# Store feature importance scores
feature_importance_df = pd.DataFrame({
    "Feature": X.columns,
    "Importance": perm_importance.importances_mean
})

# Sort features by importance
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)

# Print feature importances
print("Permutation Feature Importance:")
print(feature_importance_df.to_string(index=False))

# Select top features based on a threshold
importance_threshold = 0.01  # Example threshold
selected_features_permutation = feature_importance_df[feature_importance_df["Importance"] > importance_threshold]["Feature"].tolist()

print("\nSelected Features based on Permutation Feature Importance:")
print(selected_features_permutation)


Forward Selection (Wrapper Method)  
Forward Selection is a stepwise feature selection technique that starts with no features and iteratively adds the most important one based on model performance. The process stops when adding more features no longer improves accuracy. It helps in reducing complexity while maintaining model efficiency.

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LogisticRegression

# Define features and target variable
X = df[numerical_features + categorical_features]
y = df["NObeyesdad_Label"]  # Target variable

# Initialize the logistic regression model
logreg = LogisticRegression(max_iter=1000)

# Perform forward selection using Sequential Feature Selector
sfs = SFS(logreg, 
          k_features="best",  # Automatically selects the best number of features
          forward=True,  # Forward selection
          floating=False, 
          scoring="accuracy", 
          cv=5)  # 5-fold cross-validation

sfs.fit(X, y)

# Selected features
selected_features_forward_wrapper = list(sfs.k_feature_names_)

print("\nSelected Features using Forward Selection (Wrapper Method):")
print(selected_features_forward_wrapper)


Backward Selection (Wrapper Method)
Backward Selection starts with all features and iteratively removes the least important one based on model performance. The process continues until removing features no longer improves accuracy. It helps in eliminating irrelevant features while maintaining model efficiency.

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Define features and target variable
X = df[numerical_features + categorical_features]
y = df["NObeyesdad_Label"]  # Target variable

# Initialize logistic regression model
logreg = LogisticRegression(max_iter=1000)

# Perform backward selection using Sequential Feature Selector
sfs = SFS(logreg, 
          k_features="best", 
          forward=False,  # Backward selection
          floating=False, 
          scoring="accuracy", 
          cv=5)  # 5-fold cross-validation

sfs.fit(X, y)

# Selected features
selected_features_backward = list(sfs.k_feature_names_)

print("\nSelected Features using Backward Selection:")
print(selected_features_backward)


Recursive Feature Elimination (RFE)
RFE is a wrapper method that iteratively removes the least important features based on model performance. It starts with all features, trains the model, and removes the weakest one until the optimal subset is found. This helps in selecting the most relevant features while reducing complexity.

In [ ]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Define features and target variable
X = df[numerical_features + categorical_features]
y = df["NObeyesdad_Label"]  # Target variable

# Apply Random Fourier Transform
rft = RBFSampler(gamma=1.0, random_state=42, n_components=100)

# Define a pipeline with RFT and Logistic Regression
pipeline = Pipeline([
    ("rft", rft),
    ("logreg", LogisticRegression(max_iter=1000))
])

# Fit the model
pipeline.fit(X, y)

# Extract transformed features
X_transformed = pipeline.named_steps["rft"].transform(X)

# Display shape of transformed feature set
print("Shape of Transformed Feature Set using RFT:", X_transformed.shape)


In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# Assuming the last column is the target variable
target = df.columns[-1]
X = df.drop(columns=[target])
y = df[target]

# Add a constant column for the intercept
X = sm.add_constant(X)
a
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Forward Selection
selected_features = ['const']  # Start with only the intercept
remaining_features = list(X_train.columns)
remaining_features.remove('const')

best_model = None
best_adj_r2 = -float('inf')

while remaining_features:
    best_feature = None
    for feature in remaining_features:
        model = sm.OLS(y_train, X_train[selected_features + [feature]]).fit()
        if model.rsquared_adj > best_adj_r2:
            best_adj_r2 = model.rsquared_adj
            best_feature = feature
            best_model = model
    
    if best_feature:
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)
    else:
        break  # Stop if no improvement

print("Selected Features:", selected_features)
print(best_model.summary())


NameError: name 'df' is not defined